# Script for creating PowerPoint presentation with Cells, each slide contains: *Top-left - Analyzed Ca Transients, *Bottom-left - Analyzed Sar Shortening, *Delay, dFF/dLL and Amplitude/Baseline in the center, on the right side there are pictures from microscope: Cell before and after, and line-scans (both fluorescence and transmission images)

In [2]:
import os
import glob
import numpy as np
from PIL import Image
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
import io
from io import BytesIO
import re
from tifffile import imread, imwrite

directory_path = r'D:\My dissertation\obesity\23-08-21\Animal1.mdb'#put your directory path
#directory_path = r'C:\Users\julba\Desktop\example_for_powerpoint'
Nslice = 2048 #cut for line-scan images fro presentation


# Create a presentation object
prs = Presentation()
prs.slide_width = Inches(20)
prs.slide_height = Inches(9)

#green colormap

red_channel = np.zeros(256, dtype=np.uint8)
green_channel = np.arange(256, dtype=np.uint8)
blue_channel = np.zeros(256, dtype=np.uint8)
# Combine the red, green, and blue channels into a tuple
colormap = (red_channel, green_channel, blue_channel)

# Loop over the directories, subdirectories, and files in the directory path
for root, dirs, files in os.walk(directory_path):
    matching_file = next((f for f in files if re.search(r'\.ome\.tif$', f)), next((f for f in files if re.search(r'\.png$', f)), None))
    if matching_file is None:
        continue
    # add a slide with a blank layout
    slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(slide_layout)

    animal_folder = os.path.basename(os.path.dirname(root))
    # Print the current directory path
    print(root)
# Find the keyword (Running or Sedentary) in the root path
    root_parts = os.path.normpath(root).split(os.sep)
    keyword = ""
    for part in root_parts:
        if "Running" in part or "Sedentary" in part:
            keyword = part
            break
        
    # Extract the folder name from the root path
    folder_name = os.path.basename(root)

    # add a text box
    left = Inches(12.5)
    top = Inches(0.3578838582677165)
    width = Inches(1.0118689851268592)
    height = Inches(0.706836176727909)

    txBox = slide.shapes.add_textbox(left, top, width, height)
    tf = txBox.text_frame

    # add some text to the text box
    p = tf.add_paragraph()
    p.text = f'{animal_folder}_{folder_name}'
    # set font color
    font = p.runs[0].font
    font.color.rgb = RGBColor(0x00, 0x00, 0x00)  # Black color
    ome_files = glob.glob(os.path.join(root, "*.ome.tif"))
    png_files = glob.glob(os.path.join(root, "*.png"))
    # Check filename for placement specifications
    for ome_file in ome_files:
    # Add the image to the slide
        filename = os.path.basename(os.path.normpath(ome_file))
        protocol_match = re.search(r'protocol', filename, flags=re.IGNORECASE)
        po_match = re.search(r'Po', filename, flags=re.IGNORECASE)
        image = imread(ome_file)
        if protocol_match:
            for channel in range(2):
                single_channel_data = image[channel,:Nslice]
                image_buffer = BytesIO()
                if channel == 0:
                    imwrite(image_buffer, single_channel_data, photometric='palette', colormap=colormap)
                    img_linescan_fluo = slide.shapes.add_picture(image_buffer, left=Inches(16.64867016622922), top=Inches(0.3578838582677165), width=Inches(3.2), height=Inches(3.2))
                else:
                    imwrite(image_buffer, single_channel_data)
                    img_linescan_sar = slide.shapes.add_picture(image_buffer, left=Inches(16.64867016622922), top=Inches(4.499998906386701), width=Inches(3.2), height=Inches(3.2))
        elif po_match:
            for channel in range(2):
                single_channel_data = image[channel]
                image_buffer = BytesIO()
                if channel == 0:
                    imwrite(image_buffer, single_channel_data, photometric='palette', colormap=colormap)
                    img_po_fluo = slide.shapes.add_picture(image_buffer, left=Inches(11.069406167979002), top=Inches(4.849121828521435), width=Inches(2.5), height=Inches(2.5))
                else:
                    imwrite(image_buffer, single_channel_data)
                    img_po_sar = slide.shapes.add_picture(image_buffer, left=Inches(13.935429790026246), top=Inches(4.849121828521435), width=Inches(2.5), height=Inches(2.5))
        else:
            for channel in range(2):
                single_channel_data = image[channel]
                if single_channel_data.dtype != np.uint8:
                    array_normalized = single_channel_data / np.max(single_channel_data)
                    single_channel_data = (array_normalized * 255).astype(np.uint8)
                image_buffer = BytesIO()
                if channel == 0:
                    imwrite(image_buffer, single_channel_data, photometric='palette', colormap=colormap)
                    img_fluo = slide.shapes.add_picture(image_buffer, left=Inches(11.069406167979002), top=Inches(1.4605260279965004), width=Inches(2.5), height=Inches(2.5))
                else:
                    imwrite(image_buffer, single_channel_data)
                    img_sar = slide.shapes.add_picture(image_buffer, left=Inches(13.935429790026246), top=Inches(1.4605260279965004), width=Inches(2.5), height=Inches(2.5))

    for png_file in png_files:
        with open(png_file, 'rb') as stream:
            # Get only the file name without the directory
            filename = os.path.basename(png_file)
            good_img = False
            # Check filename for placement specifications
            if '_F_analysis' in filename:
                left, top, width, height = Inches(0.1478882327209099), Inches(0.1107884951881015), Inches(6.5), Inches(4.33)
                good_img = True
            elif '_L_analysis' in filename:
                left, top, width, height = Inches(0.10597878390201225), Inches(4.499998906386701), Inches(6.5), Inches(4.33)
                good_img = True
            elif '_L_analysis' not in filename and '_L' in filename:
                left, top, width, height = Inches(0.10597878390201225), Inches(4.499998906386701), Inches(6.5), Inches(4.33)
                good_img = True
            elif '_delay' in filename:
                left, top, width, height = Inches(6.8), Inches(0.15789479440069992), Inches(3.4736843832021), Inches(2.6052635608048993)
                good_img = True            
            elif '_dFFdLL' in filename:
                left, top, width, height = Inches(6.8), Inches(2.7897659667541555), Inches(3.83625656167979), Inches(3.0690048118985125)
                good_img = True
            elif'_ab' in filename: 
                left, top, width, height = Inches(6.8), Inches(5.930994094488189), Inches(3.83625656167979), Inches(3.0690048118985125)
                good_img = True
            # Add the image to the slide
            if good_img:
                image = slide.shapes.add_picture(stream, left, top, width, height)
                
save_directory = os.path.join(directory_path)       
os.makedirs(save_directory, exist_ok=True)

# Create the full save path
save_path = os.path.join(save_directory, "presentation_23-08-21-new.pptx")
                

    # Save the presentation
prs.save(save_path)


D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell1
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell10
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell11
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell12
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell13
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell14
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell15
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell16
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell17
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell18
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell19
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell2
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell20
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell21
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell22
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell23
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell24
D:\My dissertation\obesity\23-08-21\Animal1.mdb\Cell25
D:\My disser